# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,14.92,62,96,1.23,SH,1707273296
1,1,iqaluit,63.7506,-68.5145,-24.00,54,20,2.06,CA,1707273297
2,2,isafjordur,66.0755,-23.1240,-5.09,92,100,5.80,IS,1707273297
3,3,tura,25.5198,90.2201,15.98,68,0,1.12,IN,1707273297
4,4,el bauga,18.2620,33.9081,11.54,41,100,5.15,SD,1707273297


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points('Lng',
                                               'Lat',
                                               geo=True,
                                               tiles='OSM',
                                               size = 'Humidity',
                                               frame_width = 1000,
                                               frame_height = 800, 
                                               color = 'City')

# Display the map
humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions_df = city_data_df[(city_data_df['Cloudiness']==0) &
                                   (city_data_df['Max Temp']<27)&
                                   (city_data_df['Max Temp']>21)&
                                   (city_data_df['Wind Speed']<4.5)]
                                   

# Drop any rows with null values
ideal_conditions_df.dropna()

# Display sample data
ideal_conditions_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
76,76,margaret river,-33.9500,115.0667,25.03,36,0,3.51,AU,1707273320
190,190,nuriootpa,-34.4833,139.0000,22.90,27,0,3.89,AU,1707273124
231,231,vila velha,-20.3297,-40.2925,22.01,92,0,1.34,BR,1707273362
414,414,sangola,17.4333,75.2000,23.72,37,0,3.22,IN,1707273413
482,482,irymple,-34.2500,142.1667,23.20,35,0,0.89,AU,1707273433


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_conditions_df[['City','Country','Lat','Lng','Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = pd.Series([])

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
76,margaret river,AU,-33.9500,115.0667,36,NaN
190,nuriootpa,AU,-34.4833,139.0000,27,NaN
231,vila velha,BR,-20.3297,-40.2925,92,NaN
414,sangola,IN,17.4333,75.2000,37,NaN
482,irymple,AU,-34.2500,142.1667,35,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "filter":'',
    "bias":'',
    "apiKey":geoapify_key,
    "categories": 'accommodation.hotel'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, 'Lat']
    lng = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
margaret river - nearest hotel: Margaret River Hotel
nuriootpa - nearest hotel: The Vine Inn
vila velha - nearest hotel: Hotel Prainha
sangola - nearest hotel: No hotel found
irymple - nearest hotel: Chaffey Motor Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
76,margaret river,AU,-33.9500,115.0667,36,Margaret River Hotel
190,nuriootpa,AU,-34.4833,139.0000,27,The Vine Inn
231,vila velha,BR,-20.3297,-40.2925,92,Hotel Prainha
414,sangola,IN,17.4333,75.2000,37,No hotel found
482,irymple,AU,-34.2500,142.1667,35,Chaffey Motor Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points('Lng',
                                        'Lat', 
                                        geo=True,
                                        tiles='OSM',
                                        size='Humidity',
                                        frame_width = 1000,
                                        frame_height = 800,
                                        hover_cols=['Country','Hotel Name'],
                                       color = 'City')

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)